In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [5]:
liquidity_factors=get_all_factors(3,True)

In [10]:
final_res=pd.DataFrame()
r2=pd.Series()
rmse=pd.Series()
alpha={}
tstat={}
for sec in liquidity_factors.columns:
    ind=liquidity_factors.dropna().index
    reges=get_aqr_factors('All').loc[ind]
    reges['Market Return']=cleansed.resample(rule='m',how='last').pct_change().mean(axis=1).dropna()
    reges['const']=1
    #reges['TERM']=quandl.get('FRED/THREEFYTP10',authtoken=token).Value.resample(rule='m',how='last')
    res=sm.OLS(liquidity_factors[sec].loc[ind],reges).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    final_res[sec]=res.params
    final_res[sec+'Tstats']=res.tvalues
    r2[sec]=res.rsquared
    rmse[sec]=math.sqrt(res.mse_model)
    


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  if __name__ == '__main__':


In [12]:
final_res.T.round(4)[['const', u'Market Return','VAL', u'MOM','TSMOM','CARRY' ]]

,const,Market Return,VAL,MOM,TSMOM,CARRY
Amihud Monthly,0.0047,-0.0148,-0.3195,-0.1572,0.0407,-0.1438
Amihud MonthlyTstats,2.6576,-0.1295,-2.5307,-0.9822,0.6014,-1.3948
FHT Monthly,0.0000,0.4092,-0.1698,-0.2114,0.0414,-0.0444
FHT MonthlyTstats,0.0017,5.7063,-2.1496,-2.3543,1.0481,-0.6445
Amihud Annual,0.0051,-0.1032,-0.1430,-0.0934,0.0599,-0.2235
Amihud AnnualTstats,2.9026,-1.1494,-1.3723,-0.7386,0.9385,-2.4039
FHT Annual,0.0011,0.3008,-0.2619,-0.3279,0.1231,-0.0144
FHT AnnualTstats,0.6971,4.0922,-3.5264,-3.9083,3.0366,-0.1985


In [13]:
print final_res.T.round(2)[['const', u'Market Return','VAL', u'MOM' ,'TSMOM','CARRY']].to_latex()

\begin{tabular}{lrrrrrr}
\toprule
{} &  const &  Market Return &   VAL &   MOM &  TSMOM &  CARRY \\
\midrule
Amihud Monthly       &   0.00 &          -0.01 & -0.32 & -0.16 &   0.04 &  -0.14 \\
Amihud MonthlyTstats &   2.66 &          -0.13 & -2.53 & -0.98 &   0.60 &  -1.39 \\
FHT Monthly          &   0.00 &           0.41 & -0.17 & -0.21 &   0.04 &  -0.04 \\
FHT MonthlyTstats    &   0.00 &           5.71 & -2.15 & -2.35 &   1.05 &  -0.64 \\
Amihud Annual        &   0.01 &          -0.10 & -0.14 & -0.09 &   0.06 &  -0.22 \\
Amihud AnnualTstats  &   2.90 &          -1.15 & -1.37 & -0.74 &   0.94 &  -2.40 \\
FHT Annual           &   0.00 &           0.30 & -0.26 & -0.33 &   0.12 &  -0.01 \\
FHT AnnualTstats     &   0.70 &           4.09 & -3.53 & -3.91 &   3.04 &  -0.20 \\
\bottomrule
\end{tabular}



In [16]:
df=pd.DataFrame()
df['R2']=(r2*100).round(2)
df['rmse']=rmse.round(3)

In [19]:
print df.to_latex()

\begin{tabular}{lrr}
\toprule
{} &     R2 &   rmse \\
\midrule
Amihud Monthly &   4.00 &  0.032 \\
FHT Monthly    &  28.79 &  0.067 \\
Amihud Annual  &   7.44 &  0.040 \\
FHT Annual     &  22.94 &  0.058 \\
\bottomrule
\end{tabular}

